# barrios + cloro

In [146]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map, TileLayer
from folium.plugins import HeatMap, MarkerCluster
import geopandas as gpd
import pandas as pd
import json
import ast

## pasamos de .shp a geoDF

In [147]:
barrios_gdf = gpd.read_file("DATA/Barrios.zip")

In [63]:
barrios_gdf.crs = "EPSG:25830"

In [77]:
#barrios_gdf.crs

In [148]:
barrios_gdf.head()

,NOMDIS,NOMBRE,COD_DIS,COD_DIS_TX,BARRIO_MAY,COD_DISBAR,COD_BAR,NUM_BAR,BARRIO_MT,COD_DISB,Area_m2,geometry
0,Villaverde,San Cristóbal,17,17,SAN CRISTOBAL,172,172,2,SAN CRISTOBAL,17-2,1.082492e+06,"POLYGON ((441930.867 4466853.189, 441929.463 4..."
1,Villaverde,Butarque,17,17,BUTARQUE,173,173,3,BUTARQUE,17-3,6.375522e+06,"POLYGON ((444144.857 4464473.211, 443940.236 4..."
2,Villaverde,Los Ángeles,17,17,LOS ANGELES,175,175,5,LOS ÁNGELES,17-5,1.935271e+06,"POLYGON ((441147.728 4466374.483, 441111.738 4..."
3,Villaverde,Los Rosales,17,17,LOS ROSALES,174,174,4,LOS ROSALES,17-4,1.540747e+06,"POLYGON ((442098.826 4468081.978, 442132.060 4..."
4,Villaverde,"Villaverde Alto, Casco Histórico de Villaverde",17,17,"VILLAVERDE ALTO, CASCO HISTORICO DE VILLAVERDE",171,171,1,"VILLAVERDE ALTO, CASCO HISTÓRICO DE VILLAVERDE",17-1,9.253554e+06,"POLYGON ((440129.276 4468362.078, 440048.482 4..."


## podemos dibujar directamente los barrios (sin datos) a partir del geoDF

In [114]:
madrid_map_barr = folium.Map(location=[40.4216,-3.6780], zoom_start=12, tiles="cartodbpositron");
folium.GeoJson(data=barrios_gdf["geometry"]).add_to(madrid_map_barr);
madrid_map_barr

## pasamos de geoDF a json:

In [66]:
barrios_json = ast.literal_eval(barrios_gdf.to_json())

In [115]:
type(barrios_json)

dict

In [116]:
barrios_json.keys()

dict_keys(['type', 'features'])

In [119]:
barrios["features"][34]["properties"]["COD_BAR"]

'193'

## incluso podemos guardarlo como json... y cargarlo:

In [120]:
barrios_gdf.to_file("DATA/barrios.geojson", driver="GeoJSON") 

In [121]:
geo_json = r"DATA/barrios.geojson"
with open (geo_json) as geo_file:
    barrios = json.load(geo_file)

In [91]:
type(barrios)

dict

In [92]:
barrios.keys()

dict_keys(['type', 'crs', 'features'])

In [93]:
barrios["features"][34]["properties"]["COD_BAR"]

'193'

## cargamos el data:

In [95]:
df = pd.read_csv("DATA/dist_barr_pobl.csv")
df.sample(3)

,Distrito,Barrio,Población,Código
1,CENTRO,EMBAJADORES,47365,12
120,SAN BLAS-CANILLEJAS,AMPOSTA,8814,203
39,CHAMBERI,TRAFALGAR,24768,73


## Creamos el mapa y el coroplético:

## 1 - con el json cargado:

### a) con MÉTODO Chloropleth:

In [138]:
geo=folium.GeoJson(data=barrios_gdf["geometry"])

In [142]:
madrid_barrios = folium.Map(location=[40.4216,-3.6780], zoom_start=10, tiles="cartodbpositron");

madrid_barrios.choropleth(
    geo_data = barrios_json,
    data = df, 
    columns = ["Código", "Población"],
    key_on="feature.properties.COD_BAR", 
    fill_color = "YlGnBu",
    fill_opacity = 0.4,
    line_opacity = 1,
    legend_name = "Población total por barrios",
    smooth_factor = 1
);

In [143]:
madrid_barrios  # sale mapa pero sin nada

### b) con CLASE Chloropleth:

In [144]:
madrid_barrios = folium.Map(location=[40.4216,-3.6780], zoom_start=10, tiles="cartodbpositron");

folium.Choropleth(
    geo_data=barrios_json,
    data=df,
    columns = ["Código", "Población"],
    key_on="feature.properties.COD_BAR", 
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Población total por barrios",
    highlight=True,
).add_to(madrid_barrios);

In [145]:
madrid_barrios  # sale mapa pero sin nada

## 2 - con el json del geoDF:

### a) con MÉTODO Chloropleth:

In [123]:
madrid_barrios = folium.Map(location=[40.4216,-3.6780], zoom_start=10, tiles="cartodbpositron");

madrid_barrios.choropleth(
    geo_data = barrios_json,
    data = df, 
    columns = ["Código", "Población"],
    key_on="feature.properties.COD_BAR", 
    fill_color = "YlGnBu",
    fill_opacity = 0.4,
    line_opacity = 1,
    legend_name = "Población total por barrios",
    smooth_factor = 1
);

/home/a_romegar/anaconda3/envs/ironhack/lib/python3.8/site-packages/folium/folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [127]:
madrid_barrios;  # sale mapa pero sin nada

### b) con CLASE Chloropleth:

In [125]:
madrid_barrios = folium.Map(location=[40.4216,-3.6780], zoom_start=10, tiles="cartodbpositron");

folium.Choropleth(
    geo_data=barrios_json,
    data=df,
    columns = ["Código", "Población"],
    key_on="feature.properties.COD_BAR", 
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Población total por barrios",
    highlight=True,
).add_to(madrid_barrios);

In [128]:
madrid_barrios;  # sale mapa pero sin nada